<a href="https://colab.research.google.com/github/Nituhimani/Nituhimani/blob/chat_with_pdf/Done__Question_answering_with_multiple_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answering with multiple PDFs using DeciLM-7B-instruct, LangChain and FAISS

In [ ]:
%%capture
!pip install langchain transformers accelerate tiktoken openai gradio torch accelerate \
safetensors sentence-transformers faiss-gpu bitsandbytes pypdf typing-extensions

In [ ]:
%%capture
!pip uninstall typing-extensions --yes
!pip install typing-extensions

Downloading sample files

In [ ]:
!gdown 1GZoh1XDNbaH97HUgWwr-lCDmHq4a8yTX

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1GZoh1XDNbaH97HUgWwr-lCDmHq4a8yTX

but Gdown can't. Please check connections and permissions.


In [ ]:
!gdown 1IAMpfIir7iK-4EAI221gMMXlINzLxsNK

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1IAMpfIir7iK-4EAI221gMMXlINzLxsNK

but Gdown can't. Please check connections and permissions.


# The Processes involved in creating the chatbot

## Import required packages

In [ ]:
%%capture
!pip install PyPDF2

In [ ]:

import torch
import PyPDF2 # pdf reader
import time
from pypdf import PdfReader
from io import BytesIO
from langchain.prompts import PromptTemplate # for custom prompt specification
from langchain.text_splitter import RecursiveCharacterTextSplitter # splitter for chunks
from langchain.embeddings import HuggingFaceEmbeddings # embeddings
from langchain.vectorstores import FAISS # vector store database
from langchain.chains import RetrievalQA # qa and retriever chain
from langchain.memory import ConversationBufferMemory # for model's memoy on past conversations
from langchain.document_loaders import PyPDFDirectoryLoader # loader fo files from firectory

from langchain.llms.huggingface_pipeline import HuggingFacePipeline # pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [ ]:
# Initialize some variables

In [ ]:
CHUNK_SIZE = 1000
# Using HuggingFaceEmbeddings with the chosen embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {"device": "cuda"})

# transformer model configuration
# this massively model's precision for memory effieciency
# The models accuacy is reduced.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## The LLM in use - `DeciLM-7B-instruct`

In [ ]:
tensor_1 = torch.rand (4,4)

In [ ]:
model_id = "Deci/DeciLM-7B-instruct" # model repo id
device = 'cuda' # Run on gpu if available else run on cpu

#
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             trust_remote_code=True,
                                             device_map = "auto",
                                             quantization_config=quant_config)

# create a pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                return_full_text = True,
                max_new_tokens=200,
                repetition_penalty = 1.1,
                num_beams=5,
                no_repeat_ngram_size=4,
                early_stopping=True)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Loading the PDFs from directory with PyPDFDirectoryLoader

We load the files with PyPDFDirectoryLoader DocumentLoader which returns Document object. Document is an object with page_content and metdata. The metadata can be sources, page numbers etc.


In [ ]:
#pdf_paths = "/content/NIPS-2017-attention-is-all-you-need-Paper.pdf"

#loader = PyPDFDirectoryLoader(
    #path= pdf_paths,
    #glob="*.pdf"
#)
#documents=loader.load()

#print(len(documents))
#documents[0] # display four documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
documents = loader.load()
print(documents[0])

page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcan

In [ ]:
documents[0].page_content, documents[0].metadata

('Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly\nless ti

## Splitting the documents

We can not pass the entire documents into the language model since they have the maximum number of tokens. for instance the DeciLM-7B-instruct has 4096. Large chunks also make it hard for the model to search for relevant information from very prmpts.

So we need to split the documents into small text chunks

We split the documents with `RecursiveCharacterTextSplitter`. We sent the

* `chunk_size = 1000`(1000 characters) and

* `chunk_overlap =  200`(200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,
                                                chunk_overlap=100)

splits = text_splitter.split_documents(documents)

# length of all splits

print(f"We have, {len(splits)} chunks in memory")

We have, 40 chunks in memory


## Creating a Vector store

Now that we've got 119 text chunks in memory, we need to store and index them so that we
can search them later in our Question answering app.

We use a Vector store database for this. In this case FAISS. We embedd the text chunks and sttore them. This
helps for easier similarity check between the stored chunks and our query to the bot.


We use Hugging Face embeddings.

In [ ]:
vectorstore_db = FAISS.from_documents(splits, embeddings) # create vector db for similarity search

## Next we create a Question Answering chain.

First, we want an application that let's the user ask a question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and finally returns an answer.

To allow this, We use a `Retriever` interface which wraps an index that can return relevant documents given a string query. It uses the similarity search capabilities of a vector store to facillitate retrieval. In that case, We will convert the `vectorstore_db` we created above into a retriever.

In [ ]:

# performs a similarity check and returns the top K embeddings
# that are similar to the question’s embeddings
retriever = vectorstore_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

For instance based on our PDFs, we can get the relevant documents based on a query.

PDF 1 : "Attention is all You need"

In [ ]:
retrieved_relevant_docs = retriever.get_relevant_documents(
    "Based on the Deep unlearning pdf, what does class unlearning mean?"
)

print(f"Retrieved documents: {len(retrieved_relevant_docs)}")
f"Page content of first document:\n {retrieved_relevant_docs[0].page_content}"


Retrieved documents: 6


'Page content of first document:\n layer. arXiv preprint arXiv:1701.06538 , 2017.\n[27] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-\nnov. Dropout: a simple way to prevent neural networks from overﬁtting. Journal of Machine\nLearning Research , 15(1):1929–1958, 2014.\n[28] Sainbayar Sukhbaatar, arthur szlam, Jason Weston, and Rob Fergus. End-to-end memory\nnetworks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. Sugiyama, and R. Garnett, editors,\nAdvances in Neural Information Processing Systems 28 , pages 2440–2448. Curran Associates,\nInc., 2015.\n[29] Ilya Sutskever, Oriol Vinyals, and Quoc VV Le. Sequence to sequence learning with neural\nnetworks. In Advances in Neural Information Processing Systems , pages 3104–3112, 2014.\n[30] Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, and Zbigniew Wojna.\nRethinking the inception architecture for computer vision. CoRR , abs/1512.00567, 2015.'

PDF 2 : "Tex-to Music Generation"

In [ ]:
retrieved_relevant_docs = retriever.get_relevant_documents(
    """Based on Eight things to know about large language models pdf,
    where do much of the expense of developing an LLM go?"""
)

print(f"Retrieved documents: {len(retrieved_relevant_docs)}")
f"Page content of first document:\n {retrieved_relevant_docs[0].page_content}"

Retrieved documents: 6


'Page content of first document:\n Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the\nEnglish-to-German and English-to-French newstest2014 tests at a fraction of the training cost.\nModelBLEU Training Cost (FLOPs)\nEN-DE EN-FR EN-DE EN-FR\nByteNet [15] 23.75\nDeep-Att + PosUnk [32] 39.2 1.0·1020\nGNMT + RL [31] 24.6 39.92 2.3·10191.4·1020\nConvS2S [8] 25.16 40.46 9.6·10181.5·1020\nMoE [26] 26.03 40.56 2.0·10191.2·1020\nDeep-Att + PosUnk Ensemble [32] 40.4 8.0·1020\nGNMT + RL Ensemble [31] 26.30 41.16 1.8·10201.1·1021\nConvS2S Ensemble [8] 26.36 41.29 7.7·10191.2·1021\nTransformer (base model) 27.3 38.1 3.3·1018\nTransformer (big) 28.4 41.0 2.3·1019\nLabel Smoothing During training, we employed label smoothing of value ϵls= 0.1[30]. This\nhurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.\n6 Results\n6.1 Machine Translation\nOn the WMT 2014 English-to-German translation task, the big transformer 

PDF 3 : "APIs"

Since we have seen how the retriever is working, We will implement the  Question answeing chain using `RetrievalQA`
from Langchain. However improve user experience and the quality of the conversation with the chatbot,
we need to implement `Memory`

### Implementing memory

Large Language Models are by default `stateless` meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

For our chatbot, remembering previous interactions is important. So we will allow the DeciLM-7B-instruct LLM Lto remember previous interactions with the user.


We will use the `ConversationBufferMemory` from LangChain.

First, we create a custom prompt

### Create a custom prompt

In [ ]:
custom_prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= {context}
History = {history}
Question= {question}
Helpful Answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

## Final question answeing chain with memory

In [ ]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',
                                                   retriever = vectorstore_db.as_retriever(),
                                                   return_source_documents = True,
                                                   chain_type_kwargs = {"verbose": True,
                                                                        "prompt": prompt,
                                                                        "memory": ConversationBufferMemory(
                                                                            input_key="question",
                                                                            memory_key="history",
                                                                            return_messages=True)})



Try some queries now and see the answers

In [ ]:
query = "Based on the Deep unlearning pdf, what is deep unlearning?"
qa_chain_with_memory({"query": query})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= layer. arXiv preprint arXiv:1701.06538 , 2017.
[27] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-
nov. Dropout: a simple way to prevent neural networks from overﬁtting. Journal of Machine
Learning Research , 15(1):1929–1958, 2014.
[28] Sainbayar Sukhbaatar, arthur szlam, Jason Weston, and Rob Fergus. End-to-end memory
networks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. Sugiyama, and R. Garnett, editors,
Advances in Neural Information Processing Systems 28 , pages 2440–2448. Curran Associates,
Inc., 2015.
[29] Ilya Sutskever, O

{'query': 'Based on the Deep unlearning pdf, what is deep unlearning?',
 'result': 'Deep unlearning is a technique used in machine learning to improve the performance of a model by\nremoving or "forgetting" information that is no longer relevant to the task at hand. It involves\ntraining a model on a large amount of data, and then gradually removing or "unlearning" parts of the\nmodel that are no longer necessary for the task. This allows the model to focus on the most relevant\ninformation and improve its performance.',
 'source_documents': [Document(page_content='layer. arXiv preprint arXiv:1701.06538 , 2017.\n[27] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-\nnov. Dropout: a simple way to prevent neural networks from overﬁtting. Journal of Machine\nLearning Research , 15(1):1929–1958, 2014.\n[28] Sainbayar Sukhbaatar, arthur szlam, Jason Weston, and Rob Fergus. End-to-end memory\nnetworks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. S

Ask a follow-up question to see if the model has history of previous queries

In [ ]:
qa_chain_with_memory({"query": "What are the key challenges involved?"})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= the maximum path length between any two input and output positions in networks composed of the
different layer types.
As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requires O(n)sequential operations. In terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence
lengthnis smaller than the representation dimensionality d, which is most often the case with
sentence representations used by state-of-the-art models in machine tr

{'query': 'What are the key challenges involved?',
 'result': "The key challenges involved in deep unlearning are:\n1. Identifying which parts of the model are no longer relevant and should be removed.\n2. Determining how much information to remove and when to remove it.\n3. Ensuring that the model's performance does not degrade too much as a result of the unlearning process.",
 'source_documents': [Document(page_content='the maximum path length between any two input and output positions in networks composed of the\ndifferent layer types.\nAs noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially\nexecuted operations, whereas a recurrent layer requires O(n)sequential operations. In terms of\ncomputational complexity, self-attention layers are faster than recurrent layers when the sequence\nlengthnis smaller than the representation dimensionality d, which is most often the case with\nsentence representations used by state-of-the-art models

### To check and verify the memory/chat history

In [ ]:
print(qa_chain_with_memory.combine_documents_chain.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Based on the Deep unlearning pdf, what is deep unlearning?'), AIMessage(content='Deep unlearning is a technique used in machine learning to improve the performance of a model by\nremoving or "forgetting" information that is no longer relevant to the task at hand. It involves\ntraining a model on a large amount of data, and then gradually removing or "unlearning" parts of the\nmodel that are no longer necessary for the task. This allows the model to focus on the most relevant\ninformation and improve its performance.'), HumanMessage(content='What are the key challenges involved?'), AIMessage(content="The key challenges involved in deep unlearning are:\n1. Identifying which parts of the model are no longer relevant and should be removed.\n2. Determining how much information to remove and when to remove it.\n3. Ensuring that the model's performance does not degrade too much as a result of the unlearning process.")]) input_key=